In [ ]:
import numpy as np
from PIL import Image

from ctypes import pointer,byref

from mamj2k_ctypes import *

In [ ]:
mamj2k_path = ctypes.util.find_library('mamj2k')
print(f'Loading MAMJ2K library from {mamj2k_path}')
mj2k = cdll.LoadLibrary(mamj2k_path)

In [ ]:
mj2k.mj2k_opj_version.restype = c_char_p
mj2k.mj2k_opj_version().decode('UTF-8')

In [ ]:
def display_rgb_image(image):
    c = image.comp[0]
    w = c.w
    h = c.h
    n = w * h
    print(f'origin:({c.x0},{c.y0} size:({w},{h}) stride:({c.dx},{c.dy})')
    rgb = ( image.comp[i] for i in range(3) )
    rgb = ( ctypes.cast(c.pixels, POINTER(c_uint8 * n)).contents for c in rgb )
    rgb = [ np.array(p).reshape((h,w)) for p in rgb ]
    rgb = np.dstack(rgb)
    display(Image.fromarray(rgb))
    
def display_bw_image(comp):
    w = comp.w
    h = comp.h
    n = w * h
    print(f'origin:({comp.x0},{comp.y0} size:({w},{h}) stride:({comp.dx},{comp.dy})')
    pixels = ctypes.cast(comp.pixels, POINTER(c_uint8 * n)).contents
    pixels = np.array(pixels).reshape((h,w))
    image = Image.fromarray(pixels)
    display(image)
    
def display_image(image):
    if (image.ncomp == 3 and
        image.comp[0].rgb_compatible(image.comp[1]) and
        image.comp[0].rgb_compatible(image.comp[2])
       ):
        print("DISPLAY RGB")
        display_rgb_image(image)
    else:
        print("NOT RGB, DISPLAY COMPONENTS")
        for i in range(image.ncomp):
            print(f'COMP {i+1}')
            display_bw_image(image.comp[i])

In [ ]:
j2k_filename = 'SampleImagery/p0_07.j2k'

mj2k.mj2k_read_j2k.argtypes = [c_char_p]
mj2k.mj2k_read_j2k.restype = POINTER(MJ2K_IMAGE_T)
image_p = mj2k.mj2k_read_j2k(j2k_filename.encode('UTF-8'))
image = image_p.contents

display_image(image)

with open(j2k_filename,'rb') as j2k_file:
    j2k = j2k_file.read()
    
mj2k.mj2k_parse_j2k.argtypes = [c_char_p,c_int64]
mj2k.mj2k_parse_j2k.restype = POINTER(MJ2K_IMAGE_T)
image_p = mj2k.mj2k_parse_j2k(j2k, len(j2k))
image = image_p.contents

display_image(image)
